In [3]:
import numpy as np
import cv2 as cv

In [ ]:

# Carregar o modelo
net = cv.dnn.readNetFromCaffe(
    'colorization_deploy_v2.prototxt',
    'colorization_release_v2.caffemodel'
)

pts = np.load('pts_in_hull.npy')

# Camada de ab (cores)
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2, 313, 1, 1)

net.getLayer(class8).blobs = [pts.astype(np.float32)]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

# Carregar imagem em P&B
frame = cv.imread('images/oldphot_saopaulo.jpeg')
gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
gray = cv.cvtColor(gray, cv.COLOR_GRAY2RGB)

h, w = gray.shape[:2]
img_rs = cv.resize(gray, (224, 224))
img_lab = cv.cvtColor(img_rs, cv.COLOR_RGB2Lab)
l = img_lab[:, :, 0]

l -= 50  # Normalização para o modelo

net.setInput(cv.dnn.blobFromImage(l))
ab = net.forward()[0, :, :, :].transpose((1, 2, 0))
ab = cv.resize(ab, (w, h))

L = cv.cvtColor(frame, cv.COLOR_BGR2Lab)[:, :, 0]
colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)
colorized = cv.cvtColor(colorized.astype(np.uint8), cv.COLOR_Lab2BGR)

# Mostrar o resultado
cv.imshow("Colorized", colorized)
cv.waitKey(0)
cv.destroyAllWindows()
